# <font color=darkcyan>  Dimension reduction in Python </font>

### <font color=darkorange> SVD, Image compression, PCA, Classification, ICA</font>

### <font color=darkred>  Singular Value Decomposition applied to image compression </font>

The Singular Value Decomposition (SVD) states that for all $\mathbb{R}^{n \times d}$ matrix $A$ with rank $r$, there exist $\sigma_1\geqslant \ldots \geqslant \sigma_r>0$ such that
$$
A = \sum_{k=1}^r \sigma_k u_k v_k'\,,
$$
where $\{u_1,\ldots,u_r\}\in (\mathbb{R}^n)^r$ and $\{v_1,\ldots,v_r\}\in (\mathbb{R}^d)^r$ are two orthonormal families. The vectors $\{\sigma_1,\ldots,\sigma_r\}$ are called singular values of $A$ and $\{u_1,\ldots,u_r\}$ (resp. $\{v_1,\ldots,v_r\}$) are the left-singular (resp. right-singular) vectors of $A$.


1. If $U$ denotes the $\mathbb{R}^{n\times r}$ matrix with columns given by $\{u_1,\ldots,u_r\}$ and $V$ denotes the $\mathbb{R}^{p \times r}$ matrix with columns given by $\{v_1,\ldots,v_r\}$, then the singular value decomposition of $A$ may also be written as
$$
A = UD_rV'\,,
$$
where $D_r = \mathrm{diag}(\sigma_1,\ldots,\sigma_r)$.


2. The singular value decomposition is closely related to the spectral theorem for symmetric semipositive definite matrices. In the framework of this practical session, $A'A$ and $AA'$ are positive semidefinite such that
$$
A'A = VD_r^2V'\quad\mathrm{and}\quad AA' = UD_r^2U'\,.
$$

The numpy.linalg.svd function can be used in Python to compute the SVD of a given matrix. The output of this function are:
1. $U$ has left singular vectors in the columns ;
2. sigma is rank 1 numpy array with singular values ;
3. $V$ has right singular vectors in the rows.

In [ ]:
# ignore warnings for better clarity (may not be the best thing to do)...
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from PIL import Image
# Image.open is used to open the input picture (with any .jpg or .png)
img = Image.open('./seals.jpg')
img

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Image converted into a numpy array or matrix
img_mat       = np.array(list(img.getdata(band=0)), float)
img_mat.shape = (img.size[1], img.size[0])
img_mat       = np.matrix(img_mat)
# SVD can then be applied to the matrix img_mat

In [ ]:
# write a function with input the path of an image "path_image" and an integer "k" 
# and return in gray scale the reconstructed picture with the first k singular values
def svd_decomposition(path_image,k):
    img           = Image.open(path_image)
    img_mat       = np.array(list(img.getdata(band=0)), float)
    img_mat.shape = (img.size[1], img.size[0])
    img_mat       = np.matrix(img_mat)
    
    # Perform Singular Value Decomposition
    U, sigma, V = np.linalg.svd(img_mat)
    print('Size left singular eigenvectors  ' + str(np.shape(U)))
    print('Size right singular eigenvectors  ' + str(np.shape(V)))
    print('Size eigenvalues matrix ' + str(np.shape(sigma)))
    
    # Image reconstruction
    reconstimg = np.matrix(U[:, :k]) * np.diag(sigma[:k]) * np.matrix(V[:k, :])
    
    fig = plt.figure(1)
    plt.plot(sigma[0:12]*100/np.sum(sigma))
    plt.title("Normalized values of the singular values (in %)")
    fig = plt.figure(2)
    plt.title("Image reconstruction with %g singular values"%k)
    plt.imshow(reconstimg, cmap='gray')
    plt.axis('off')

In [ ]:
# Image reconstruction using interact to analyze the influence of the number of singular values
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
interact(svd_decomposition,path_image='./seals.jpg',k=(1,100));

### <font color=darkred>  Principal Component Analysis </font>

#### <font color=darkred>Application of the SVD to Principal Component Analysis</font>

Let $(X_i)_{1\leqslant i\leqslant n}$ be i.i.d. random variables in $\mathbb{R}^d$ and consider the matrix $X\in\mathbb{R}^{n\times d}$ such that the $i$-th row of $X$ is the observation $X'_i$. Let $\Sigma_n$ be the empirical covariance matrix (data are assumed to be centered for simplicity, this can be done manually):
$$
\Sigma_n = n^{-1}\sum_{i=1}^n X_i X'_i\,.
$$
Principal Component Analysis  aims at reducing the dimensionality of the observations $(X_i)_{1\leqslant i \leqslant n}$ using a "compression" matrix $W\in \mathbb{R}^{p\times d}$ with $p\leqslant d$ so that for each $1\leqslant i \leqslant n$, $WX_i$ ia a low dimensional representation of $X_i$. The original observation may then be partially recovered using another matrix $U\in \mathbb{R}^{d\times p}$. Principal Component Analysis computes $U$ and $W$ using the least squares approach:
$$
(U_{\star},W_{\star}) \in \hspace{-0.5cm}\underset{(U,W)\in \mathbb{R}^{d\times p}\times \mathbb{R}^{p\times d}}{\mathrm{argmin}} \;\sum_{i=1}^n\|X_i - UWX_i\|^2\,, 
$$

Let $(U_{\star},W_{\star})\in \mathbb{R}^{d\times p}\times \mathbb{R}^{p\times d}$ be a solution to this problem. Then, it can be proved that the columns of $U_{\star}$ are orthonormal and $W_{\star} = U_{\star}'$. Therefore, solving the optimization problem boils down to computing
$$
U_{\star} \in \hspace{-0.5cm}\underset{U\in \mathbb{R}^{d\times p}\,,\, U'U = I_n}{\mathrm{argmax}} \hspace{-.4cm}\{ \mathrm{trace}(U'\Sigma_nU)\}\,.
$$
Let $\{\vartheta_1,\ldots,\vartheta_d\}$ be orthonormal eigenvectors associated with the eigenvalues $\lambda_1\geqslant \ldots \geqslant \lambda_d$ of $\Sigma_n$. Then a solution is given by the matrix $U_{\star}$ with columns $\{\vartheta_1,\ldots,\vartheta_p\}$ and $W_{\star} = U_{\star}'$.

#### <font color=darkred>Principal Component Analysis as an optimization problem</font>

For any dimension $1\leqslant p \leqslant  d$, let $\mathcal{F}_d^p$ be the set of all vector suspaces of $\mathbb{R}^d$ with dimension $p$. Principal Component Analysis computes a linear span $V_d$ such as
$$
V_p \in \underset{V\in \mathcal{F}_d^p}{\mathrm{argmin}} \;\sum_{i=1}^n\|X_i - \pi_V(X_i)\|^2\,, 
$$
where $\pi_V$ is the orthogonal projection onto the linear span $V$. Consequently, $V_1$ is a solution if and only if $v_1$ is solution to:
$$
v_1 \in \underset{v \in \mathbb{R}^d\,;\, \|v\|=1}{\mathrm{argmax}} \sum_{i=1}^n   \langle X_i, v \rangle^2\,.
$$
For all $2\leqslant p \leqslant d$, following the same steps, it can be proved that  a solution is given by $V_p = \mathrm{span}\{v_1, \ldots, v_p\}$ where
$$
v_1 \in \underset{v\in \mathbb{R}^d\,;\,\|v\|=1}{\mathrm{argmax}} \sum_{i=1}^n\langle X_i,v\rangle^2 \quad\mbox{and for all}\;\; 2\leqslant k \leqslant p\;,\;\; v_k \in \underset{\substack{v\in \mathbb{R}^d\,;\,\|v\|=1\,;\\ v\perp v_1,\ldots,v\perp v_{k-1}}}{\mathrm{argmax}}\sum_{i=1}^n\langle X_i,v\rangle^2\,. 
$$

As $V_p = \mathrm{span}\{\vartheta_1, \ldots, \vartheta_p\}$, for all $1\leqslant i\leqslant n$,
$$
\pi_{V_p}(X_i) = \sum_{k=1}^p \langle X_i,\vartheta_k\rangle \vartheta_k  = \sum_{k=1}^p (X'_i \vartheta_k)\vartheta_k = \sum_{k=1}^p c_k(i)\vartheta_k\,,
$$
where for all $1\leqslant k \leqslant p$, the $k$-th principal component is defined as $c_k = X\vartheta_k$. Therefore the $k$-th principal component is the vector whose components are the coordinates of each $X_i$, $1\leqslant i\leqslant n$, relative to the basis $\{\vartheta_1, \ldots, \vartheta_p\}$ of $V_p$.

In [ ]:
# Build a data set with n=100 bi-dimensional i.i.d. data with centered Gaussian distribution
X = np.dot(np.random.normal(0,1,[2,2]), np.random.normal(0,1,[2,200])).T
plt.scatter(X[:, 0], X[:, 1], s = 10)

In [ ]:
# perform a PCA with one component using PCA(n_components=1) and the function fit.
from sklearn.decomposition import PCA 
pca = PCA(n_components=1)
pca.fit(X)
print('The principal component is:')
print(pca.components_)

print('The explained variance is %g'%pca.explained_variance_)
print('The associated singular value is %g'%pca.singular_values_)

# Apply the dimensionality reduction on X
# X_pca contains the coordinates of each data in the space generated by the principal components
X_pca = pca.transform(X)

# in this case pca.components_[k] contains the coordinates of the k-th principal component in
# the original space (here the usual Euclidian plane). In a general case pca.components_[k] is a 
# d-dimensional vector.
# X_pca[i] contains the coordinates of the i-th data in the vector space generated by the principal 
# components.
# Therefore, X_pca[i] is a vector with n_components entries. 

In a general case, when $\mathrm{n\_components} = p$ for all $1\leqslant i\leqslant n$ and all $1\leqslant k \leqslant p$, the
projection of $X_i$ in the space generated by the principal components is:

$$
\pi_{V_p}(X_i) = \sum_{k=1}^{p}X_{\mathrm{pca}}[i]_k \times \mathrm{pca.components\_}[k]\,.
$$

In [ ]:
# transform the reduced data set in the original space
# X_inverse[i,:] contains the coordinates of the projection of Xi in the original space
X_inverse = pca.inverse_transform(X_pca)
plt.scatter(X[:, 0], X[:, 1], alpha=0.2, s=10)
plt.scatter(X_inverse[:, 0], X_inverse[:, 1], alpha=0.8, s=10)

In [ ]:
# Data must be centered and scaled when the variables have different units, such as height (cm) and weight (kg). 
# This prevents the main directions from being governed by one or more variables with a higher variance than the 
# other variables. See the illustration below.
# When the variables have the same unit, the reduction should be considered on a case-by-case basis. 
# Sometimes, it is not advised to force variances to be equal (for example, with respect to grades, we may want to discriminate 
# more against students in relation to a subject whose grades vary more)

In [ ]:
import pandas as pd
X = np.dot(np.random.normal(0,1,[3,3]), np.random.normal(0,1,[3,200])).T
X[:,0] = 10*X[:,0] + 15 
pca = PCA(n_components=2)
pca.fit(X)
X_pca = pca.transform(X)
principalDf = pd.DataFrame(data = X_pca, columns = ['Principal Component 1', 'Principal Component 2'])

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('PCA with two principal components', fontsize = 20)
ax.scatter(principalDf['Principal Component 1'], principalDf['Principal Component 2'], s = 10)
ax.grid()

print('The first principal component explains %f of the variance'%pca.explained_variance_ratio_[0])  
print('The second principal component explains %f of the variance'%pca.explained_variance_ratio_[1]) 
plt.axis('equal');

In [ ]:
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X)
principalComponents = pca.fit_transform(X_scaled) 

principalDf = pd.DataFrame(data = principalComponents, columns = ['Principal Component 1', 'Principal Component 2'])

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('PCA with two principal components', fontsize = 20)
ax.scatter(principalDf['Principal Component 1'], principalDf['Principal Component 2'], s = 10)
ax.grid()
plt.axis('equal');

print('The first principal component explains %f of the variance'%pca.explained_variance_ratio_[0])  
print('The second principal component explains %f of the variance'%pca.explained_variance_ratio_[1]) 

In [ ]:
# If you perform a standardized PCA on a huge number of variables: the percentage of variability of the two first dimensions
# is not necessarily small. In the case of independent Gaussian variables, the percentage of variability decreases with the 
# dimension. If the data set is made by a copy of the same variable in each column, the percentage variability of the two first
# dimensions will then be 100% regardless of the number of columns.

In [ ]:
# a scaled PCA has been performed with 4 data sets:

<img src="./PCA_cor.png" width="500" height="700">

In [ ]:
# it is possible to link these correlation matrices with the PCA results.
# The value of the first eigenvalue approximately gives the number of correlated 
# variables explained by the first dimension.

# The matrix A seems to be a block diagonal matrix with blocks of size $3x3$ and $2x2$. 
# The block of size $2x2$ is close to the matrix with only ones (which is of rank 1) thus 
# one eigenvalue must be close to zero. This corresponds to PCA 4.

# The matrix B is has all its entries near one, thus it is close to a matrix of
# rank one. If this matrix was of rank one, the variance could be explained with
# the first component only. In that case, the inertia associated with the first
# component would be $5$ and $0$ for the other components. This corresponds to PCA 1.

# The matrix D is close to the identity. The PCA applied to the identity would
# give the same inertia to all components (inertia close to 1). This corresponds
# to PCA 2.

# The matrix C is similar to B but the correlation are weaker. Thus the first
# component has a high inertia. This corresponds to PCA 3.

In [ ]:
# in the graph below, it is possible to give roughly the percentage of variability explained by the first dimension
# and by the first plane.

<img src="./AnaDo_ACP_exo_Graphe_var2.jpg" width="600" height="600">

In [ ]:
# the number of variables aligned with the first two dimensions should be divided by the total number of variables. 
# The first dimension explains 44% of the variability and the first plane 77%. The percentage of explained variance is equal to the sum of square norms of
# projection of variables along the first axis. Variables V5, V6, V9 are orthogonal to the first axis and variables V7 and V8 are not well represented by
# the first plan (and thus are orthogonal to this plan and in particular to the first axis). Roughly, the norm of V1, V2, V3 and V4 is one which leads to a
# percentage of variance for the first axis equal to $4/9$. Similarly, for the first plan, we have a percentage of variance 
# equal to $7/9$ since the norm of $V5$ and $V9$ is close to one.

## <font color=darkred> PCA on a real data set </font>

In [ ]:
# this section focuses on a data set where each observation is a global confort given by an employee 
# in a large building (related to temperature, noise, CO2 level).

In [ ]:
# import the data about confort at workplace "data_confort.csv"
df = pd.read_csv('data_confort.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

In [ ]:
# Data rescaling using min_max_scaler
from sklearn.preprocessing import MinMaxScaler
array          = df.values 
min_max_scaler = MinMaxScaler()
x_scaled       = min_max_scaler.fit_transform(array)
X              = (pd.DataFrame(x_scaled, columns = df.columns))
X.head()

In [ ]:
# each category is defines as follows if Class_i = 1 it means that the confort is affected either by 
# CO2 level, humidity, temperature or noise level at the time of the data collection.
# if all Class_i = 0, this means that the confort is good enough (category 0).
categories = []
for i in df.iloc[:,-4:].values:
    if np.array_equal(i, [0,0,0,0]):
        categories.append(0) 
    
    elif np.array_equal(i, [0,0,0,1]):
        categories.append(1) 
    
    elif np.array_equal(i, [0,0,1,0]):
        categories.append(2) 
    
    elif np.array_equal(i, [0,1,0,0]):
        categories.append(3) 
    
    else :
        categories.append(4) 

In [ ]:
# categories Class_1, Class_2, etc. are replaced by a single category in {0,1,2,3,4} 
X['categories'] = np.array(categories)
X = X.drop(["Class_1", "Class_2", "Class_3", "Class_4"],1)

In [ ]:
# means of the features grouped by categories using the function groupby
X.groupby(["categories"]).mean().plot(kind='bar',figsize=(15,7))

In [ ]:
# display the correlations of the features and the categories using heatmap
import seaborn as sns
corr = X.corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, cmap = 'Blues')

In [ ]:
# compute the mean table by category. 
# the function apply is used to compute the mean value of each feature in each category with respect 
# to its mean value in the best confort category.
mean_table = X.groupby(['categories']).mean().T
index_list = mean_table.index.values
mean_table = pd.DataFrame(mean_table.values, index=index_list,columns=['5','4','3','2','1'])
mean_table["4 % 5"] = mean_table.apply(lambda x : (x.iloc[1] - x.iloc[0]) / x.iloc[0]*100,axis=1 )
mean_table["3 % 5"] = mean_table.apply(lambda x : (x.iloc[2] - x.iloc[0]) / x.iloc[0]*100,axis=1 )
mean_table["2 % 5"] = mean_table.apply(lambda x : (x.iloc[3] - x.iloc[0]) / x.iloc[0]*100,axis=1 )
mean_table["1 % 5"] = mean_table.apply(lambda x : (x.iloc[4] - x.iloc[0]) / x.iloc[0]*100,axis=1 )
mean_table.head()

In [ ]:
# plot the associated values
mean_table.iloc[:,-4:].groupby(index_list).mean().plot(kind='bar',figsize=(15,7))

<font color=darkred> Provide boxplots of some features among categories</font>

In [ ]:
# some boxplots displaying the distribution of a feature within each category.
# This may help to detect useless features for classification... or a feature highly impacted by the category.
X.boxplot('Feat_11', by=['categories'],figsize=(15,7))

In [ ]:
X.boxplot('Feat_3', by=['categories'],figsize=(15,7))

In [ ]:
# compute a PCA with two components (after data scaling)
from sklearn import preprocessing 
norm_pca = preprocessing.scale(X)
n_comp = 6
pca = PCA(n_components = n_comp)

x = norm_pca[:,:-1]

principalComponents = pca.fit_transform(x)
principalDf         = pd.DataFrame(data = principalComponents, columns = ['Princ. Comp. ' + str(i) for i in range(n_comp)])

finalDf = pd.concat([principalDf, X.categories], axis = 1)

In [ ]:
finalDf.head()

In [ ]:
pca.explained_variance_ratio_
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance');

## <font color=darkred> Use of PCA before Machine learning applications (classification here)</font>

In [ ]:
# provide a clustering of the in the plane given by the first two principal components (for illustration purposes)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets_names = ['5','4','3','2','1']
targets = [0,1,2,3,4]

for target in targets:
    indicesToKeep = finalDf['categories'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'Princ. Comp. 0'], finalDf.loc[indicesToKeep, 'Princ. Comp. 1'],s=10)

ax.legend(targets_names)
ax.grid()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance');

<font color=darkred> Split the dataset using 70% of the data to learn a classifier and 30% to test the classifier performance.</font>

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import time

In [ ]:
X_train , X_test, Y_train , Y_test = train_test_split(X.iloc[:,:-1] , X.iloc[:,-1] , test_size = 0.3)

In [ ]:
X_train.head()

In [ ]:
Y_train.head()

<font color=darkred> Use a PCA with 2 components before learning a Support Vector Machine classifier (with SVC) on the training set. Compute the classification score.</font>

In [ ]:
start_t = time.time()
pca = PCA(n_components=2)
pca.fit(X_train)

X_train_transform = pca.transform(X_train)
X_test_transform  = pca.transform(X_test)

clf = SVC()
clf.fit(X_train_transform, Y_train)
end_t = time.time()
print('Classification score %g'%clf.score(X_test_transform, Y_test))
print('Computational time %g sec.'%(end_t-start_t))

<font color=darkred> Compare the results for several number of components using cross validation.</font>

In [ ]:
test_score = np.zeros(10)
comp_time  = np.zeros(10)
for n_comp in np.arange(2,12):
    start_t = time.time()
    pca     = PCA(n_components=n_comp)

    pca.fit(X.iloc[:,:-1])

    X_transform          = pca.transform(X.iloc[:,:-1])
    kfold                = StratifiedKFold(n_splits=5, shuffle=True) 
    cv_results           = cross_val_score(clf, X_transform, X["categories"], cv=kfold)
    comp_time[n_comp-2]  = time.time()-start_t
    test_score[n_comp-2] = cv_results.mean()

In [ ]:
figure = plt.figure(figsize = (10, 6))
plt.subplot(1,2,1)
plt.plot(np.arange(2,12),test_score, '-')
plt.xlabel('Number of dimensions')
plt.ylabel('Cross validation score')
plt.subplot(1,2,2)
plt.plot(np.arange(2,12),comp_time, '-')
plt.xlabel('Number of dimensions')
plt.ylabel('Computational time (s)')
plt.tight_layout()

## <font color=darkred> Independent component analysis (ICA)</font>

In [ ]:
from scipy import signal
time_steps = np.linspace(0,150,300)

# Sources
S = np.array([signal.sawtooth(time_steps),
              np.sin(3*time_steps)]).T

A = np.array([[0.5, 3.0],
              [1.2, 1.6]])

# Observed signal
X = S.dot(A).T

plt.plot(time_steps, X[0], lw = 1)
plt.plot(time_steps, X[1], lw = 1)
plt.title('observed signals', fontsize = 15)
 
fig = plt.figure()
plt.plot(time_steps, S, lw = 1)
plt.title('Unknown sources', fontsize = 15)

Preprocessing proceeds in two steps. The first step amounts to `centering` the data. For all $1\leqslant i \leqslant d$,

$$\tilde x_{ij} = x_{ij}-{\frac {1}{n}}\sum _{\ell = 1}^nx_{i\ell}\,.$$

Then, a `linear transformation is used to transform data into a new dataset with unit covariance matrix`.  A standard approach to do so is to perform an eigenvalue decomposition of the empirical covariance matrix $\Sigma$ of the centered data $\tilde X = (\tilde x_{ij})_{1\leqslant i\leqslant d, 1\leqslant j\leqslant n}$:  
$$\Sigma = P \Delta P^{-1}\,,$$
where $\Delta$ is the diagonal matrix containing all the eigenvalues of $\Sigma$. The unit covariance covariance data is the given by:  
$$
\widehat X= \Delta^{-1/2}P^{T}\tilde X\,.
$$

In [ ]:
# Preprocessing data

# Center data
Xmean     = np.mean(X, axis=1, keepdims=True)
Xcentered =  X - Xmean

# Transform data into a new dataset with unit covariance matrix
# In the case of a square matrix, an eigenvalue decomposition is equivalent to a SVD.
Sigma         = np.cov(Xcentered)
U, S, V       = np.linalg.svd(Sigma)
deltainv      = np.diag(1.0 / np.sqrt(S))
weight_matrix = np.dot(U, np.dot(deltainv, U.T))
Xrescaled     = np.dot(weight_matrix, Xcentered)

In [ ]:
def fast_ICA(signals, W, m, nb_it):
    W = W_init.copy()
    for c in range(m):        
        wc = W[c, :].copy()
        wc = wc.reshape(m, 1)
        wc = wc / np.sqrt(np.sum(wc ** 2))
        for i in range(nb_it):
            # Compute intermediate weights to "maximize non-Gaussianity"
            wX = np.dot(wc.T, signals)
            
            w1 = np.tanh(wX).T
            w2 = (1 - np.square(np.tanh(wX)))

            # Weight update
            w_new = (signals * w1.T).mean(axis = 1) - np.mean(w2) * wc.squeeze()
            w_new = w_new - np.dot(np.dot(w_new, W[:c].T), W[:c])
            w_new = w_new / np.sqrt(np.sum(w_new ** 2))
            
            wc = w_new

        W[c, :] = wc.T
    return W

In [ ]:
# Initialize weights
m      = 2
W_init = np.random.rand(m, m)
nb_it  = 100

W = fast_ICA(Xrescaled, W_init, m, nb_it)

recoveredX = Xrescaled.T.dot(W.T)
recoveredX = (recoveredX.T - Xmean).T

In [ ]:
fig = plt.figure()
plt.plot(time_steps, S[:,0], label='First source', lw = 1)
plt.plot(time_steps, S[:,1], label='Second source', lw = 1)
plt.plot(time_steps, recoveredX[:,0], '--', label='First recovered signal', lw = 1)
plt.plot(time_steps, recoveredX[:,1], '--', label='Second recovered signal', lw = 1)